In [8]:
from workflows import DYStudiesProcessor

import json
from coffea.processor import (
    run_uproot_job,
    iterative_executor, 
    futures_executor,
    NanoAODSchema
)

with open("dummy_samples.json") as f:
    samples = json.load(f)

import json
import os.path as osp
metaconditions_file = "metaconditions/Era2017_legacy_v1.json"
with open(metaconditions_file) as f:
    metaconditions = json.load(f)    

dystudies = DYStudiesProcessor(metaconditions, False, True, './outputs/')

ModuleNotFoundError: No module named 'workflows'

In [ ]:
histos = run_uproot_job(
    samples,
    'Events',
    dystudies,
    executor=futures_executor,
    executor_args={"schema": NanoAODSchema,"workers": 4},
    )

In [ ]:
x = ak.ones_like(ak.flatten(events.Photon.pt))

In [ ]:
import pandas as pd
import pyarrow as pa
import awkward as ak

x = pd.read_parquet('./outputs/')

#y = ak.from_arrow(pa.Table.from_pandas(x))

In [ ]:
x

In [ ]:
# di-photon MVA
import xgboost as xg

min_diphoton_mass = 100
max_diphoton_mass = 180

model = xg.Booster()
model.load_model('aux-data/altDiphoModel_coffea.model')

# get the number of diphotons per row
# and save for re-wrapping xgb outputs
counts = ak.num(diphotons, axis=1)

# extract diphoton vars into flat lists
dipho_leadIDMVA = ak.flatten(diphotons["0"].mvaID)
dipho_subleadIDMVA = ak.flatten(diphotons["1"].mvaID)
dipho_lead_ptoM = ak.flatten(diphotons["0"].pt / diphotons.mass)
dipho_sublead_ptoM = ak.flatten(diphotons["1"].pt / diphotons.mass)
dipho_lead_eta = ak.flatten(diphotons["0"].eta)
dipho_sublead_eta = ak.flatten(diphotons["1"].eta)

diphoVars  = ['dipho_leadIDMVA', 'dipho_subleadIDMVA', 'dipho_lead_ptoM', 
              'dipho_sublead_ptoM', 'dipho_leadEta', 'dipho_subleadEta', 
              'CosPhi', 'vtxprob', 'sigmarv', 'sigmawv']
allVars = diphoVars + ["dipho_mass"]

f = uproot.open('data/ggH_powheg_UL_2017.root')
tree = f['vbfTagDumper/trees/ggh_125_13TeV_GeneralDipho']
arrays = tree.arrays(allVars, how=dict)

mask = (  (arrays["dipho_mass"]> min_diphoton_mass) & (arrays["dipho_mass"]< max_diphoton_mass) 
        & (arrays["dipho_leadIDMVA"]>-0.9) & (arrays["dipho_subleadIDMVA"]>-0.9) 
        & (arrays["dipho_lead_ptoM"]>0.333) & (arrays["dipho_sublead_ptoM"]>0.25))

x = np.column_stack((ak.to_numpy(arrays[var][mask]) for var in diphoVars))[:100]

print(x.shape)

diphoMatrix = xg.DMatrix(x, feature_names=diphoVars)


y = model.predict(diphoMatrix)

In [5]:
import xgboost
import warnings
from functools import partial
from pprint import pprint
import json
import os.path as osp
metaconditions_file = "metaconditions/Era2017_legacy_xgb_v1.json"
with open(metaconditions_file) as f:
    metaconditions = json.load(f)
    
print(metaconditions["PhoIdInputCorrections"])

data_kind = "mc"

def wrapped_xgb(array, model, scale, center, variables):
    asDMatrix = xgboost.DMatrix(array)
    return model.predict(asDMatrix)

def create_evaluator(weights, scale=None, center=None, variables=None, **kwargs):
    try:
        model = xgboost.Booster()
        model.load_model(weights)
        return partial(wrapped_xgb, model=model, scale=scale, center=center, variables=variables)
    except xgboost.core.XGBoostError:
        raise ValueError(f"could not find: {weights}")

class ChainedQuantileRegression:
    def __init__(
        self, 
        corrections_summary, 
        SS_variables, 
        correctShowerShapes=True, 
        correctIsolations=True, 
        correctPreshower=False,
    ):
        with open(corrections_summary, "r") as f:
            cq_config = json.load(f)
        
        self.transforms = {}
        
        for vargroup, varlist in cq_config.items():
            self.transforms[vargroup] = {}
            for varname, regions in varlist.items():
                for region, steps in regions.items():
                    if not correctIsolations and vargroup == "isolations":
                        continue
                    if not correctShowerShapes and vargroup == "shower_shapes":
                        continue
                    self.transforms[vargroup][varname] = {}
                    if vargroup == "isolations":
                        self.transforms[vargroup][varname][region] = {}
                        for stepname, config in steps.items():
                            keys = list(config.keys())
                            print(stepname, ":", keys)
                            if "variables" in keys:
                                config["variables"] = [var.split(":=")[-1].strip() for var in config["variables"]]
                            if f"weights_data" in keys and "weights_mc" in keys:
                                config["weights"] = config["weights_data"]
                                self.transforms[vargroup][varname][region][f"{stepname}_data"] = (
                                    create_evaluator(**config)
                                )
                                config["weights"] = config["weights_mc"]
                                self.transforms[vargroup][varname][region][f"{stepname}_mc"] = (
                                    create_evaluator(**config)
                                )
                            else:
                                self.transforms[vargroup][varname][region][stepname] = (
                                    create_evaluator(**config)
                                )
                    elif vargroup == "shower_shapes":
                        self.transforms[vargroup][varname][region] = []
                        self.transforms[vargroup][varname][region].append(
                            create_evaluator(**steps)
                        )
                    else:
                        raise Exception(
                            f"{vargroup} is not understood by ChainedQuantileRegression"
                        )
        self.ssvs = [ssv.split(":=")[-1].strip() for ssv in SS_variables]
    
    def apply(self, events):
        pass
            
test = ChainedQuantileRegression(**metaconditions["PhoIdInputCorrections"])



{'corrections_summary': 'aux-data/Taggers/PhoIdInputsCorrections/corrections_summary_2017_Legacy_xgb.json', 'SS_variables': ['f0 := pt', 'f1 := eta', 'f2 := phi', 'f3 := fixedGridRhoAll', 'f4 := sieip', 'f5 := s4', 'f6 := r9', 'f7 := phiWidth', 'f8 := sieip', 'f9 := etaWidth']}
peak2tail : ['variables', 'weights']
peak_tail_clfs : ['weights_data', 'weights_mc', 'variables']
morphing : ['variables', 'scale', 'weights', 'center']
peak2tail : ['variables', 'weights']
peak_tail_clfs : ['weights_data', 'weights_mc', 'variables']
morphing : ['variables', 'scale', 'weights', 'center']
chIso_peak2tail : ['variables', 'weights']
peak_tail_clfs : ['weights_data', 'weights_mc', 'variables']
chIsoWorst_peak2tail : ['variables', 'weights']
chIsoWorst_morphing : ['variables', 'scale', 'weights', 'center']
chIso_morphing : ['variables', 'scale', 'weights', 'center']
chIso_peak2tail : ['variables', 'weights']
peak_tail_clfs : ['weights_data', 'weights_mc', 'variables']
chIsoWorst_peak2tail : ['variabl

In [17]:
test.transforms["isolations"]["phoIso"]["EB"]["morphing"]

functools.partial(<function wrapped_xgb at 0x7f7499f0f9d0>, model=<xgboost.core.Booster object at 0x7f7499f27ca0>, scale=0.08328429357760053, center=0.04449992030025607, variables=['pt', 'superCluster.eta', 'phi', 'global.rho', 'pfPhoIso03'])

In [ ]:
import pickle
import gzip

x = pickle.dumps(test)

In [ ]:
import lz4.frame

print(len(x))
print(len(lz4.frame.compress(x, compression_level=9)))

In [10]:
import time

import xgboost
from functools import partial
from pprint import pprint
import json
import os.path as osp

import awkward as ak
import numpy as np
import vector
from coffea import nanoevents

vector.register_awkward()

metaconditions_file = "metaconditions/Era2017_legacy_xgb_v1.json"
with open(metaconditions_file) as f:
    metaconditions = json.load(f)
    
print(metaconditions["flashggDiPhotonMVA"])
var_order = metaconditions["flashggDiPhotonMVA"]["inputs"]

# get some events to play with
events = nanoevents.NanoEventsFactory.from_root("../data/step2_HggNano_test.root").events()
diphotons = ak.combinations(events.Photon, 2)

p4s = diphotons.slot0 + diphotons.slot1
diphotons["pt"] = p4s.pt
diphotons["eta"] = p4s.eta
diphotons["phi"] = p4s.phi
diphotons["mass"] = p4s.mass
diphotons["charge"] = p4s.charge
diphotons = ak.with_name(diphotons, "PtEtaPhiMCandidate")

bdt_vars = {}

bdt_vars["dipho_leadIDMVA"] = diphotons.slot0.mvaID
bdt_vars["dipho_subleadIDMVA"] = diphotons.slot1.mvaID
bdt_vars["dipho_leadEta"] = diphotons.slot0.eta
bdt_vars["dipho_subleadEta"] = diphotons.slot1.eta
bdt_vars["dipho_lead_ptoM"] = diphotons.slot0.pt / diphotons.mass
bdt_vars["dipho_sublead_ptoM"] = diphotons.slot1.pt / diphotons.mass

# calculate sigma_wv
def calc_displacement(photons, events):
    x = photons.x_calo - events.PV.x
    y = photons.y_calo - events.PV.y
    z = photons.z_calo - events.PV.z
    return ak.zip({"x": x, "y": y, "z": z}, with_name="Vector3D")

tic = time.monotonic()
v_lead = calc_displacement(diphotons.slot0, events)
v_sublead = calc_displacement(diphotons.slot1, events)

p_lead = v_lead.unit() * diphotons.slot0.energyRaw
p_lead["energy"] = diphotons.slot0.energyRaw
p_lead = ak.with_name(p_lead, "Momentum4D")
p_sublead = v_sublead.unit() * diphotons.slot1.energyRaw
p_sublead["energy"] = diphotons.slot1.energyRaw
p_sublead = ak.with_name(p_sublead, "Momentum4D")

sech_lead = 1.0 / np.cosh(p_lead.eta)
sech_sublead = 1.0/ np.cosh(p_sublead.eta)
tanh_lead = np.cos(p_lead.theta)
tanh_sublead = np.cos(p_sublead.theta)

cos_dphi = np.cos(p_lead.deltaphi(p_sublead))

numerator_lead = sech_lead * (sech_lead * tanh_sublead - tanh_lead * sech_sublead * cos_dphi)
numerator_sublead = sech_sublead * (sech_sublead * tanh_lead - tanh_sublead * sech_lead * cos_dphi)

denominator = 1.0 - tanh_lead * tanh_sublead - sech_lead * sech_sublead * cos_dphi

add_reso = 0.5 * (-np.sqrt(2.0) * events.BeamSpot.sigmaZ / denominator) * (numerator_lead/p_lead.mag + numerator_sublead/p_sublead.mag)

dEnorm_lead = diphotons.slot0.energyErr / diphotons.slot0.energy
dEnorm_sublead = diphotons.slot1.energyErr / diphotons.slot1.energy

sigma_m = 0.5 * np.sqrt(dEnorm_lead**2 + dEnorm_sublead**2)
sigma_wv = np.sqrt(add_reso**2 + sigma_m**2)
toc = time.monotonic()

print("var calc", toc - tic)


vtx_prob = ak.full_like(sigma_m, 0.999) # !!!! placeholder !!!!

bdt_vars["CosPhi"] = cos_dphi
bdt_vars["vtxprob"] = vtx_prob
bdt_vars["sigmarv"] = sigma_m
bdt_vars["sigmawv"] = sigma_wv

# create the diphoton BDT
diphotonMVA = xgboost.Booster()
diphotonMVA.load_model(metaconditions["flashggDiPhotonMVA"]["weightFile"])

counts = ak.num(diphotons, axis=-1)
bdt_inputs = np.column_stack([ak.to_numpy(ak.flatten(bdt_vars[name])) for name in var_order])

tic = time.monotonic()
tempmatrix = xgboost.DMatrix(bdt_inputs, feature_names=var_order)
toc = time.monotonic()
print("dmatrix", toc - tic)

tic = time.monotonic()
scores = diphotonMVA.predict(tempmatrix)
toc = time.monotonic()
print("predict", toc - tic)

diphotons["bdt_score"] = ak.unflatten(scores, counts)


{'weightFile': 'aux-data/Taggers/STXSmodels/altDiphoModel_coffea.json', 'version': 'xgb', 'inputs': ['dipho_leadIDMVA', 'dipho_subleadIDMVA', 'dipho_lead_ptoM', 'dipho_sublead_ptoM', 'dipho_leadEta', 'dipho_subleadEta', 'CosPhi', 'vtxprob', 'sigmarv', 'sigmawv']}
var calc 0.26253880001604557
dmatrix 0.02345369989052415
predict 0.012677599908784032


In [ ]:
events.Photon.

In [ ]:
scores.shape[0]

In [ ]:
import uproot
import xgboost
import numpy as np
import awkward as ak

diphoVars  = ['dipho_leadIDMVA', 'dipho_subleadIDMVA', 'dipho_lead_ptoM', 
              'dipho_sublead_ptoM', 'dipho_leadEta', 'dipho_subleadEta', 
              'CosPhi', 'vtxprob', 'sigmarv', 'sigmawv']
allVars = diphoVars + ["dipho_mass"]

f = uproot.open('../data/ggH_powheg_UL_2017.root')
tree = f['vbfTagDumper/trees/ggh_125_13TeV_GeneralDipho']
arrays = tree.arrays(allVars, how=dict)

mask = (  (arrays["dipho_mass"]>100.) & (arrays["dipho_mass"]<180.) 
        & (arrays["dipho_leadIDMVA"]>-0.9) & (arrays["dipho_subleadIDMVA"]>-0.9) 
        & (arrays["dipho_lead_ptoM"]>0.333) & (arrays["dipho_sublead_ptoM"]>0.25))

x = np.column_stack((ak.to_numpy(arrays[var][mask]) for var in diphoVars))[:100]

print(x.shape)

diphoMatrix = xgboost.DMatrix(x, feature_names=diphoVars)

model = xgboost.Booster()
model.load_model('aux-data/altDiphoModel_coffea.model')
y = model.predict(diphoMatrix)

print(type(x))
print(type(y))

In [ ]:
events.behavior["__events_factory__"]._partition_key.replace('/', '_') + '.parquet'

In [ ]:
events.metadata

In [ ]:
metaconditions

In [ ]:
from workflows import taggers

In [ ]:
dir(taggers)

In [ ]:
import awkward as ak

x = ak.Array([[1.0], [2.0, 3.0], [4.0, 5.0, 6.0]])



In [ ]:
counts = ak.num(x, axis=1)
ak.unflatten([0, 0] * ak.flatten(x)[:, None], counts)

In [ ]:
import numpy as np
x = np.array([0,1,2])
y = np.array([3,4,5])

np.stack((x,y), axis=1)

In [ ]:
ak.Array([1,2,3,4])[ak.min(ak.Array([[1,-1], []]), axis=1)]

In [ ]:
str(['abc', 1, 5.0])

In [ ]:
eval("['abc', 1, 5.0]")

In [ ]:
import sys
sys.maxsize

In [ ]:
(None or 1.0)